In [ ]:
%%bash
pip install -r ../../requirements.txt
pip install transformers


### Objective
Evaluate zero-shot and few-shot prompting for aspect-aware sentiment.


In [ ]:
%%bash
set -e
python - <<'PY'
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, pandas as pd

model_name = 'mistralai/Mistral-7B-Instruct-v0.2'  # replace with a locally available instruction model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map='auto')

prompt_tmpl = (
    "You are grading sentiment about the {aspect}. "
    "Answer with one label: positive, negative, or neutral.\n"
    "Comment: {text}\nSentiment:"
)

df = pd.read_csv('../../data/teacher_course.csv')
sample = df.iloc[0]
prompt = prompt_tmpl.format(aspect=sample['aspect'], text=sample['comments'])
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
output = model.generate(**inputs, max_new_tokens=5, temperature=0.0)
print(tokenizer.decode(output[0], skip_special_tokens=True))
PY


Extend the prompt with k-shot labeled examples to run few-shot evaluations and compute accuracy or macro F1 across the dataset.
